In [ ]:

# Dynamic Pricing for Urban Parking Lots
# =============================================================================

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# =============================
# 1. Data Loading and Preprocessing
# =============================


try:
    df = pd.read_csv("/content/dataset (1).csv")
except FileNotFoundError:
    print("Error: The dataset file 'dataset (1).csv' was not found.")
    exit()

# --- Feature Engineering ---


df["timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + ' ' + df["LastUpdatedTime"], format='%d-%m-%Y %H:%M:%S')


df = df.rename(columns={
    "SystemCodeNumber": "LotID",
    "TrafficConditionNearby": "Traffic",
    "IsSpecialDay": "SpecialDay",
    "QueueLength": "Queue"
})


traffic_map = {"low": 1, "medium": 2, "high": 3, "heavy": 4}
df["Traffic"] = df["Traffic"].map(traffic_map).fillna(1)


print("Dataset Shape:", df.shape)
print("Columns:", df.columns)
print("Data Head:\n", df.head())


# =============================
# 2. Model 1: Baseline Linear Model
# =============================

def baseline_price(prev_price, occupancy, capacity, alpha=2):
    """
    Calculates the next price based on a simple linear function of the occupancy rate.
    This model serves as a basic benchmark for comparison.

    Args:
        prev_price (float): The price at the previous timestep.
        occupancy (int): The current number of vehicles in the lot.
        capacity (int): The maximum capacity of the lot.
        alpha (float): A sensitivity parameter to control the rate of price change.

    Returns:
        float: The calculated new price.
    """

    occ_rate = occupancy / capacity if capacity > 0 else 0
    return prev_price + alpha * occ_rate


# =============================
# 3. Model 2: Demand-Based Model
# =============================

def demand_based_price(base_price, occupancy, capacity, queue, traffic, special, vehicle_type,
                       alpha=0.5, beta=0.3, gamma=0.2, delta=0.5,
                       epsilon_map={"car": 0.2, "bike": 0.1, "truck": 0.4, "cycle": 0.05},
                       lam=0.5):
    """
    Calculates a price based on a multi-factor demand function.
    This model considers real-time conditions like traffic, queues, and special events.

    Args:
        base_price (float): The starting base price for the lot.
        occupancy (int): Current number of vehicles.
        capacity (int): Maximum lot capacity.
        queue (int): Number of vehicles waiting to enter.
        traffic (int): Numerical representation of nearby traffic.
        special (int): Boolean indicator for a special day (holiday, event).
        vehicle_type (str): The type of vehicle entering ('car', 'bike', etc.).
        alpha, beta, gamma, delta, lam (float): Weights for the different demand factors.
        epsilon_map (dict): A mapping of vehicle types to their corresponding weight.

    Returns:
        float: The calculated demand-based price, bounded between 0.5x and 2x the base price.
    """
    occ_rate = occupancy / capacity if capacity > 0 else 0
    vehicle_weight = epsilon_map.get(vehicle_type, 0.2)


    demand = alpha * occ_rate + beta * queue - gamma * traffic + delta * special + vehicle_weight
    demand = max(0, demand)


    price = base_price * (1 + lam * demand)


    return np.clip(price, 0.5 * base_price, 2 * base_price)


# =============================
# 4. Model 3: Competitive Pricing Model
# =============================

def competitive_price(lot_id, lots_df, base_price=10):
    """
    Adjusts the price based on the prices of nearby competitor lots.
    This adds a layer of strategic, location-aware pricing.

    Args:
        lot_id (str): The ID of the current parking lot.
        lots_df (pd.DataFrame): A DataFrame containing the current state of all lots.
        base_price (float): The base price, used for setting price bounds.

    Returns:
        float: The competitively adjusted price.
    """

    lots_df = lots_df.copy()

    #
    current = lots_df.loc[lots_df['LotID'] == lot_id].iloc[0]
    lat, lon = current["Latitude"], current["Longitude"]
    my_price = current["price"]

    # --- Competitor Analysis ---

    def dist(row):
        return geodesic((lat, lon), (row["Latitude"], row["Longitude"])).km


    lots_df["dist"] = lots_df.apply(dist, axis=1)
    competitors = lots_df[(lots_df["dist"] > 0) & (lots_df["dist"] < 0.5)]


    if not competitors.empty:
        avg_comp_price = competitors["price"].mean()

        # Business Logic: Adjust price based on occupancy and competitor pricing.
        # If our lot is not full and we are more expensive, lower the price to attract customers.
        if (current["Occupancy"] / current["Capacity"] < 0.8) and (my_price > avg_comp_price):
            my_price *= 0.9
        # If our lot is almost full and we are cheaper, we can increase the price.
        elif (current["Occupancy"] / current["Capacity"] > 0.8) and (my_price < avg_comp_price):
            my_price *= 1.1

    # Ensure the final price remains within the predefined bounds.
    return np.clip(my_price, 0.5 * base_price, 2 * base_price)


# =============================
# 5. Simulation Engine
# =============================

def run_simulation(df):
    """
    Loops through the dataset chronologically to simulate the dynamic pricing process.
    """
    results = []
    # Initialize the price for each lot to a default base price of $10.
    prev_prices = {lot: 10 for lot in df["LotID"].unique()}

    # Sort the data by time to ensure the simulation is chronological.
    df_sorted = df.sort_values(by="timestamp").reset_index(drop=True)

    # --- Main Simulation Loop ---
    # Iterate through each recorded event in the dataset.
    for t, row in df_sorted.iterrows():
        lot_id = row["LotID"]
        prev_price = prev_prices[lot_id]

        # Calculate the price using the baseline model.
        p1 = baseline_price(prev_price, row["Occupancy"], row["Capacity"])

        # Calculate the price using the demand-based model.
        p2 = demand_based_price(10, row["Occupancy"], row["Capacity"],
                                row["Queue"], row["Traffic"], row["SpecialDay"],
                                row["VehicleType"])

        # Store the results for this timestep.
        results.append({
            "timestamp": row["timestamp"], "LotID": lot_id,
            "BaselinePrice": p1, "DemandPrice": p2,
            "Latitude": row["Latitude"], "Longitude": row["Longitude"],
            "Occupancy": row["Occupancy"], "Capacity": row["Capacity"]
        })

        # Update the price for the next iteration of the baseline model.
        prev_prices[lot_id] = p1

    results_df = pd.DataFrame(results)

    # --- Apply Competitive Model ---
    # This is a simplified approach where the competitive adjustment is applied
    # after the main simulation based on the latest state of each lot.
    latest_prices = results_df.groupby('LotID').last().reset_index()
    latest_prices['price'] = latest_prices['DemandPrice']  # Use demand price as the base for comparison.

    # Calculate the competitive price for each lot and apply it to the results.
    results_df['CompetitivePrice'] = results_df['DemandPrice']
    for _, row in latest_prices.iterrows():
        comp_price = competitive_price(row['LotID'], latest_prices)
        results_df.loc[results_df['LotID'] == row['LotID'], 'CompetitivePrice'] = comp_price

    return results_df


# Run the simulation with the preprocessed data.
sim_results = run_simulation(df)
print("\nSimulation Results Head:\n", sim_results.head())


# =============================
# 6. Visualization with Bokeh
# =============================

# Initialize Bokeh for interactive plotting in a notebook environment.
output_notebook()

# Select a single lot to visualize its pricing dynamics over time.
lot_id_to_plot = df["LotID"].unique()[0]
lot_data = sim_results[sim_results["LotID"] == lot_id_to_plot]

# Create a Bokeh ColumnDataSource for efficient data handling.
source = ColumnDataSource(lot_data)

# Create a new plot with a datetime axis.
p = figure(x_axis_type="datetime", title=f"Pricing Simulation for Lot {lot_id_to_plot}",
           width=800, height=400)

# Add line renderers for each of the three pricing models.
p.line(x="timestamp", y="BaselinePrice", source=source, color="blue", legend_label="Baseline Price")
p.line(x="timestamp", y="DemandPrice", source=source, color="green", legend_label="Demand Price")
p.line(x="timestamp", y="CompetitivePrice", source=source, color="red", legend_label="Competitive Price")

# Customize the plot's appearance.
p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"

# Display the plot.
show(p)

Dataset Shape: (18368, 13)
Columns: Index(['ID', 'LotID', 'Capacity', 'Latitude', 'Longitude', 'Occupancy',
       'VehicleType', 'Traffic', 'Queue', 'SpecialDay', 'LastUpdatedDate',
       'LastUpdatedTime', 'timestamp'],
      dtype='object')
Data Head:
    ID        LotID  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0  BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1  BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2  BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3  BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4  BHMBCCMKT01       577  26.144536  91.736172        150        bike   

   Traffic  Queue  SpecialDay LastUpdatedDate LastUpdatedTime  \
0      1.0      1           0      04-10-2016        07:59:00   
1      1.0      1           0      04-10-2016        08:25:00   
2      1.0      2           0      04-10-2016        08:59:00   
3      1.0      2   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
